## Arrays Functions in PySpark

PySpark DataFrames can contain array columns. You can think of a PySpark array column in a similar way to a Python list. Arrays can be useful if you have data of a variable length. They can be tricky to handle, so you may want to create new rows for each element in the array, or change them to a string.

There are many functions for handling arrays. Here we will just demonstrate a few of them. For a full list, take a look at the [PySpark documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.array.html).

Note that since Spark 3.0, arrays are supported in sparklyr, although they are not covered in this article. The relevant [sparklyr functions](https://spark.rstudio.com/packages/sparklyr/latest/reference/) begin `hof_` (*higher order function*), e.g. [`hof_transform()`](https://spark.rstudio.com/packages/sparklyr/latest/reference/hof_transform.html)

### Creating a DataFrame with arrays

You will encounter arrays most frequently when reading in data. Spark can read parquet files that contain array columns. For this example, we will create a small DataFrame manually with an array column. To do this, simply create the DataFrame in the usual way, but supply a Python list for the column values to [`spark.createDataFrame()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html). The lists do not have to have the same number of elements.

If using a schema to create the DataFrame, import [`ArrayType()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.ArrayType.html) or use `array<type>` if using [DDL notation](../spark-overview/data-types), which is `array<string>` in this example.

The example used here will use champions of the Wimbledon tennis tournament. The score for a tennis match is often listed by individual sets, which can be displayed as an array. This array will be of variable length, as the match stops once someone wins two sets in women's matches and three in men's, meaning the array can vary between two and five elements. As the tournament was cancelled in 2020, these entries will be `null`.

In [1]:
from pyspark.sql import SparkSession, functions as F
    
spark = (SparkSession.builder.master("local[2]")
         .appName("arrays")
         .getOrCreate())

champions = spark.createDataFrame([
    [2017, "Gentlemen's Singles", "Federer", "Čilić", ["6-3", "6-1", "6-4"]],
    [2018, "Gentlemen's Singles", "Djokovic", "Anderson", ["6-2", "6-2", "7-6"]],
    [2019, "Gentlemen's Singles", "Djokovic", "Federer", ["7-6", "1-6", "7-6", "4-6", "13-12"]],
    [2020, "Gentlemen's Singles", None, None, None], #No tournament in 2020
    [2021, "Gentlemen's Singles", "Djokovic", "Berrettini", ["6-7", "6-4", "6-4", "6-3"]],
    [2017, "Ladies' Singles", "Muguruza", "V Williams", ["7-5", "6-0"]],
    [2018, "Ladies' Singles", "Kerber", "S Williams", ["6-3", "6-3"]],
    [2019, "Ladies' Singles", "Halep", "S Williams", ["6-2", "6-2"]],
    [2020, "Ladies' Singles", None, None, None], #No tournament in 2020]
    [2021, "Ladies' Singles", "Barty", "Plíšková", ["6-3", "6-7", "6-3"]],
    ], ["tournament_year", "event", "champion", "runner_up", "final_score"])

champions.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+
|tournament_year|event              |champion|runner_up |final_score                |
+---------------+-------------------+--------+----------+---------------------------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|
|2020           |Gentlemen's Singles|null    |null      |null                       |
|2021           |Gentlemen's Singles|Djokovic|Berrettini|[6-7, 6-4, 6-4, 6-3]       |
|2017           |Ladies' Singles    |Muguruza|V Williams|[7-5, 6-0]                 |
|2018           |Ladies' Singles    |Kerber  |S Williams|[6-3, 6-3]                 |
|2019           |Ladies' Singles    |Halep   |S Williams|[6-2, 6-2]                 |
|2020           |Ladies' Singles    |null    |null    

Use [`.dtypes`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.dtypes.html) to confirm that `final_score` is a string array:

In [2]:
champions.dtypes

[('tournament_year', 'bigint'),
 ('event', 'string'),
 ('champion', 'string'),
 ('runner_up', 'string'),
 ('final_score', 'array<string>')]

### Get length of array: `F.size()`

[`F.size()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.size.html) returns the number of elements in the array. This returns `-1` for `null` values. In this example, we can see how many sets were played in each match:

In [3]:
champions_size = champions.withColumn("total_sets", F.size("final_score"))
champions_size.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+----------+
|tournament_year|event              |champion|runner_up |final_score                |total_sets|
+---------------+-------------------+--------+----------+---------------------------+----------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |3         |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |3         |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|5         |
|2020           |Gentlemen's Singles|null    |null      |null                       |-1        |
|2021           |Gentlemen's Singles|Djokovic|Berrettini|[6-7, 6-4, 6-4, 6-3]       |4         |
|2017           |Ladies' Singles    |Muguruza|V Williams|[7-5, 6-0]                 |2         |
|2018           |Ladies' Singles    |Kerber  |S Williams|[6-3, 6-3]                 |2         |
|2019           |Ladies' Singl

### Convert array to string: `F.concat_ws()`

You may be familiar with [`F.concat_ws()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.concat_ws.html) to concatenate string columns. It can also be used with arrays. `null` values will be mapped to an empty string.

The first argument is the separator, followed by the columns to concatenate.

In [4]:
champions_concat = (champions
      # Traditional use of concat_ws: concatenate strings
      .withColumn("match", F.concat_ws(" v ", "champion", "runner_up"))
      # Can also use with arrays
      .withColumn("final_score_string", F.concat_ws(", ", "final_score"))
      .select("tournament_year", "event", "match", "final_score", "final_score_string"))

champions_concat.show(truncate=False)

+---------------+-------------------+---------------------+---------------------------+-------------------------+
|tournament_year|event              |match                |final_score                |final_score_string       |
+---------------+-------------------+---------------------+---------------------------+-------------------------+
|2017           |Gentlemen's Singles|Federer v Čilić      |[6-3, 6-1, 6-4]            |6-3, 6-1, 6-4            |
|2018           |Gentlemen's Singles|Djokovic v Anderson  |[6-2, 6-2, 7-6]            |6-2, 6-2, 7-6            |
|2019           |Gentlemen's Singles|Djokovic v Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|7-6, 1-6, 7-6, 4-6, 13-12|
|2020           |Gentlemen's Singles|                     |null                       |                         |
|2021           |Gentlemen's Singles|Djokovic v Berrettini|[6-7, 6-4, 6-4, 6-3]       |6-7, 6-4, 6-4, 6-3       |
|2017           |Ladies' Singles    |Muguruza v V Williams|[7-5, 6-0]                 |7

### Convert string to array: `F.split()`

The opposite of `F.concat_ws()` is [`F.split()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.split.html). This example uses the separator `", "` (as above), but it can also handle regexes.

Note that the empty strings (that were originally `null` values in the array column) have been mapped to empty arrays.

In [5]:
champions_array = (champions_concat
                   .withColumn("final_score_array",
                               F.split("final_score_string", ", ")))

champions_array.drop("final_score").show(truncate=False)

+---------------+-------------------+---------------------+-------------------------+---------------------------+
|tournament_year|event              |match                |final_score_string       |final_score_array          |
+---------------+-------------------+---------------------+-------------------------+---------------------------+
|2017           |Gentlemen's Singles|Federer v Čilić      |6-3, 6-1, 6-4            |[6-3, 6-1, 6-4]            |
|2018           |Gentlemen's Singles|Djokovic v Anderson  |6-2, 6-2, 7-6            |[6-2, 6-2, 7-6]            |
|2019           |Gentlemen's Singles|Djokovic v Federer   |7-6, 1-6, 7-6, 4-6, 13-12|[7-6, 1-6, 7-6, 4-6, 13-12]|
|2020           |Gentlemen's Singles|                     |                         |[]                         |
|2021           |Gentlemen's Singles|Djokovic v Berrettini|6-7, 6-4, 6-4, 6-3       |[6-7, 6-4, 6-4, 6-3]       |
|2017           |Ladies' Singles    |Muguruza v V Williams|7-5, 6-0                 |[7-

### Create a new row for each array element: `F.explode()` and `F.explode_outer()`

[`F.explode()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html) is used to create a new row for each element in the array. It can be much easier to handle the data from the array in this format.

In this example, `F.explode()` extracts the results of the individual sets, so the new column is called `set_score`.

In [6]:
champions_explode = champions.withColumn("set_score",
                                         F.explode("final_score"))

champions_explode.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+---------+
|tournament_year|event              |champion|runner_up |final_score                |set_score|
+---------------+-------------------+--------+----------+---------------------------+---------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-3      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-1      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-4      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |7-6      |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|7-6      |
|2019           |Gentlemen's Singles|Djo

[`F.explode_outer()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode_outer.html) works in the same way, but retains `null` values:

In [7]:
champions_explode_outer = champions.withColumn("set_score",
                                               F.explode_outer("final_score"))

champions_explode_outer.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+---------+
|tournament_year|event              |champion|runner_up |final_score                |set_score|
+---------------+-------------------+--------+----------+---------------------------+---------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-3      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-1      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |6-4      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |7-6      |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|7-6      |
|2019           |Gentlemen's Singles|Djo

### Explode with position: `F.posexplode()` and `F.posexplode_outer()`

[`F.posexplode()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.posexplode.html) returns two columns, the position of the element within the array as well as the element itself. In common with Python lists, this starts at zero.

Note that you have to use [`.select()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html) with [`.alias()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.alias.html) rather than [`.withColumn()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html), due to the function returning two columns. `.select("*")` selects all the existing columns, similar to `SELECT *` in SQL.

In [8]:
champions_posexplode = (champions
                        .select("*",
                                F.posexplode("final_score").alias("set_no", "set_score"))
                        # Add 1 to set number, as index starts from 0
                        .withColumn("set_no", F.col("set_no") + 1))

champions_posexplode.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+------+---------+
|tournament_year|event              |champion|runner_up |final_score                |set_no|set_score|
+---------------+-------------------+--------+----------+---------------------------+------+---------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |1     |6-3      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |2     |6-1      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |3     |6-4      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |1     |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |2     |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |3     |7-6      |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 

Combine with [`.filter()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html) to extract just the result of the first set:

In [9]:
champions_posexplode.filter(F.col("set_no") == 1).show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+------+---------+
|tournament_year|event              |champion|runner_up |final_score                |set_no|set_score|
+---------------+-------------------+--------+----------+---------------------------+------+---------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |1     |6-3      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |1     |6-2      |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 4-6, 13-12]|1     |7-6      |
|2021           |Gentlemen's Singles|Djokovic|Berrettini|[6-7, 6-4, 6-4, 6-3]       |1     |6-7      |
|2017           |Ladies' Singles    |Muguruza|V Williams|[7-5, 6-0]                 |1     |7-5      |
|2018           |Ladies' Singles    |Kerber  |S Williams|[6-3, 6-3]                 |1     |6-3      |
|2019           |Ladies' Singles    |Halep   |S Williams|[6-2, 6-2]      

Use [`F.posexplode_outer()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.posexplode_outer.html) to retain the `null` values:

In [10]:
champions_posexplode_outer = (champions
                              .select("*",
                                      F.posexplode_outer("final_score").alias("set_no", "set_score"))
                              # Add 1 to set number, as index starts from 0
                              .withColumn("set_no", F.col("set_no") + 1))

champions_posexplode_outer.show(truncate=False)

+---------------+-------------------+--------+----------+---------------------------+------+---------+
|tournament_year|event              |champion|runner_up |final_score                |set_no|set_score|
+---------------+-------------------+--------+----------+---------------------------+------+---------+
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |1     |6-3      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |2     |6-1      |
|2017           |Gentlemen's Singles|Federer |Čilić     |[6-3, 6-1, 6-4]            |3     |6-4      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |1     |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |2     |6-2      |
|2018           |Gentlemen's Singles|Djokovic|Anderson  |[6-2, 6-2, 7-6]            |3     |7-6      |
|2019           |Gentlemen's Singles|Djokovic|Federer   |[7-6, 1-6, 7-6, 

### Further Resources

Spark at the ONS Articles:
- [Data Types in Spark](../spark-overview/data-types)

PySpark Documentation
- [`F.array()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.array.html)
- [`spark.createDataFrame()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html)
- [`ArrayType()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.ArrayType.html)
- [`.dtypes`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.dtypes.html)
- [`F.size()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.size.html)
- [`F.concat_ws()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.concat_ws.html)
- [`F.split()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.split.html)
- [`F.explode()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html)
- [`F.explode_outer()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode_outer.html)
- [`F.posexplode()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.posexplode.html)
- [`.select()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html)
- [`.alias()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.alias.html)
- [`.withColumn()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html)
- [`.filter()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html)
- [`F.posexplode_outer()`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.posexplode_outer.html)

[sparklyr Documentation](https://spark.rstudio.com/packages/sparklyr/latest/reference/):
- [`hof_transform()`](https://spark.rstudio.com/packages/sparklyr/latest/reference/hof_transform.html)

### Acknowledgements

Thanks to [Johannes Hechler](https://github.com/JohannesHechler) for his material on PySpark arrays.